In [ ]:
import numpy as np
from scipy.spatial.distance import euclidean
import _ucrdtw
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
import seaborn as sns
from sklearn.cluster import SpectralClustering
import nibabel as nib
from sklearn import preprocessing

raw_data = np.load('./fmri_preprocessed_2017_09_18.npy')
mask = np.load('/home/chris/projects/rfmri/Data/pig_2/brandon/mask_2017_09_18.npy')
anatomical = nib.load('/home/chris/projects/rfmri/Data/pig_2/coregistered_anatomy.nii').get_data()
x,y,z = mask.nonzero() #~16,000
slice_mask = ((z >= 7) & (z <= 16))
slice_7_17_data = raw_data[slice_mask] #~12,000

# Call a new scaler to scale only slice 7-16
slice_7_17_data = np.transpose(slice_7_17_data)
scaler = preprocessing.StandardScaler().fit(slice_7_17_data.astype(float))
scaled_7_17 = scaler.transform(slice_7_17_data)
scaled_7_17 = np.transpose(scaled_7_17)
scaled_7_17 = np.expand_dims(scaled_7_17, axis=2)

distance_matrix = np.zeros((scaled_7_17.shape[0],scaled_7_17.shape[0]))
print(distance_matrix.shape)

#Calculate a dtw distance matrix in a really dumb and slow way
for i in tqdm(range(scaled_7_17.shape[0])):
    for j in range(scaled_7_17.shape[0]):
        loc, dist = _ucrdtw.ucrdtw(scaled_7_17[i], scaled_7_17[j], 0.05, True)
        distance_matrix[i,j] = dist
        
np.save('./dtw_matrix_7_17_different_scaling.npy', distance_matrix)

/home/chris/projects/rfmri/pig/envi/lib/python3.5/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int16 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
  0%|          | 0/12340 [00:00<?, ?it/s]

(12340, 12340)


 63%|██████▎   | 7811/12340 [6:58:13<3:38:38,  2.90s/it] 

In [116]:
# # This was nice though
# blah = np.identity(5)
# print(np.delete(blah, (4), axis=1))

# x,y,z = mask.nonzero()
# slice_mask = ((z >= 7) & (z <= 16))
# meow = np.where(np.logical_not(slice_mask))
# new_one = np.delete(np.delete(dtw_sim, meow[0], axis=0), meow[0], axis=1)
# new_one.shape

# clustering = SpectralClustering(n_clusters=3, n_jobs=4, affinity='precomputed').fit_predict(new_one)
# np.save('dtw_7_17_3_clusters.npy', clustering)

[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  0.]]


(12340, 12340)

In [4]:
dtw2 = np.load('./dtw_matrix_7_17_different_scaling.npy')

In [6]:
dtw2.shape


(12340, 12340)